In [ ]:
from arcsim_expert_python import *
import wrinkle2
import util
import cv2
import matplotlib.pyplot as plt

expert = arcsim_expert()
remesh_fag = False
render_image = True
MOVEMENT_THRESH = 1e-5
BETA = 0.98

expert.create_sheet(1,5,2,10,"./python_sheet",[0,0,-9.81],[0,0,0],0,0,remesh_fag)
expert.setup("./python_sheet/sheet.json")

import time
path = "./"+time.strftime("%m%d-%H%M")

x = 1.0 #width of the cloth 
y = 2.0 #length of the cloth
delta = 0.01 # controller gain, multiplier
nr_frame = 100 # number of frames each setup consists
nr_pass = 100 # number of different passes (setups)
expert_controller = expert.expert_flat #expert type

if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

# initialize the positions
handles=[[0,0,0],[x,0,0],[0,y,0],[x,y,0]]
# frame_id
frame_id=0
tt_pos = np.array([])
tt_feat = np.array([])
# add log at each run
tt_log = np.array([])

for p in range(nr_pass):
    print("TURN p:",p)
    hands=np.zeros((2,3))
    # pick two random positions for hands, which have at most x (won't break the cloth)
    while True:
        # random from [0,0,0] [x,0,0], which is the first two handles
        hands[0] = [random.uniform(-x/4,x/4),random.uniform(-x/4,x/4),random.uniform(-x/4,x/4)]
        hands[1] = [random.uniform(-x/4,x/4)+x,random.uniform(-x/4,x/4),random.uniform(-x/4,x/4)]
        if np.linalg.norm(np.subtract(hands[0],hands[1])) < x:
            break
    expert.advance()
    handles = expert.apply_hand(handles, hands, delta)
    expert.set_handle(handles)
    # save images
    filename = path+"/%04i"%frame_id;
    expert.save_frame(filename, render_image)
    
    frame_id=frame_id+1
    print(filename)
    
    
    BETA_p = BETA**p
    # start each run
    for i in range(nr_frame):
        # get the visual feature
        im = cv2.imread(filename+'.png')
        feat = wrinkle2.xhist(im)
        print(handles)
        expert_pos = expert_controller(handles,x,y)
        # update the positions
        pos = np.array(expert_pos).ravel()
        tt_pos = np.vstack((tt_pos,pos)) if tt_pos.size else pos
        # update the features
        tt_feat = np.vstack((tt_feat,feat)) if tt_feat.size else feat
        np.savez(path+'/data',tt_pos = tt_pos, tt_feat = tt_feat)
        
        expert.advance()
        handles_prev = handles
        # apply hands
        handles = expert.apply_hand(handles, hands, delta)
        # apply the robot controllers positions
                    
        if p>0 and random.random()>BETA_p:
            pred_pos = model.predict(feat.reshape(1,-1))
            pred_pos = pred_pos.reshape((2,3))
            handles = expert.apply_expert(handles, pred_pos, delta)
        else:
            handles = expert.apply_expert(handles, expert_pos, delta)
            
        expert.set_handle(handles)
        # save images
        filename = path+"/%04i"%frame_id;
        expert.save_frame(filename, render_image)
        frame_id=frame_id+1
        
        # determine whether to terminate the run
        d = np.array(handles_prev).ravel() - np.array(handles).ravel()
        d = sum(d*d)
        print("run:",p,"frame:",i,"movement delta:",d)
        if d<MOVEMENT_THRESH:
            break;
        
    # begin training
    from sklearn import linear_model
    # begin training
    model = linear_model.Lasso(1e-6)
    model.fit(tt_feat, tt_pos[:,-6:])

In [ ]:
import util
import cv2
import wrinkle2
import numpy as np
# build the initial classifier
folder = '1027-1318'
filelist = util.get_filelist(folder,'png')
tt_pos = np.load(folder+'/tt_pos.npy')
tt_feat = np.array([])
print(tt_pos)expert_pos
for fn in filelist:
    print(fn)
    # get the visual feature
    im = cv2.imread(fn)
    feat = wrinkle2.xhist(im)
    tt_feat = np.vstack((tt_feat,feat)) if tt_feat.size else feat

print(tt_feat.shape)
np.savez(folder+'/data',tt_pos = tt_pos, tt_feat = tt_feat)

# begin training
from sklearn import linear_model
model = linear_model.Lasso(1e-7)
model.fit(tt_feat, tt_pos)
print(tt_feat.shape,tt_pos.shape,len(tt_feat))

for i in range(len(tt_feat)):
    x = tt_feat[i,:].reshape(1,-1)
    target = model.predict(x)
    print(target-tt_pos[i,:])


In [55]:
import util
import cv2
import wrinkle2
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# build the initial classifier
folder = '1101-2316'

data = np.load(folder+'/data.npz')
tt_feat = data['tt_feat']
tt_pos = data['tt_pos']

X = tt_feat
y = tt_pos

# begin training
train_size = int(len(tt_feat)/2)
print(train_size)
X_train = X[:train_size,:]
y_train = y[:train_size,-6:]
X_test = X[train_size:,:]
y_test = y[train_size:,:]

for p0 in range(4,8):
    alpha = 10**(-p0)
    model = linear_model.Lasso(alpha=alpha)
    
    model.fit(X_train, y_train)
    print("alpha:",alpha, "score:", model.score(X_test, y_test))

for p0 in range(3,10):
    max_depth = p0
    model = RandomForestRegressor(max_depth=max_depth, random_state=2)
    
    model.fit(X_train, y_train)
    print("depth:",max_depth, "score:", model.score(X_test, y_test))    

# y_pred = model.predict(X)
# for i in range(y_test.shape[1]):
#     plt.plot(range(len(y_pred)),y_pred[:,i])
#     plt.plot(range(len(y)),y[:,i])
#     plt.show()
    


922
alpha: 0.0001 score: 0.318250561376
alpha: 1e-05 score: 0.41799190768


/playpen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha: 1e-06 score: 0.688218655171
alpha: 1e-07 score: 0.534814029735
depth: 3 score: 0.237266922239
depth: 4 score: 0.264656977437
depth: 5 score: 0.278121870921
depth: 6 score: 0.307583644676
depth: 7 score: 0.289747626501
depth: 8 score: 0.274975170477
depth: 9 score: 0.278465380356
